# Homework 5

> I pledge my honor that I have abided by the Stevens Honor System - Joshua Schmidt 3/27/21

- Nowadays, technology companies often use recommender algorithms to recommend products, music, movie, etc. The recommender algorithm is really useful and helps all those companies make a huge profit.
- Now, after learning the factorial matrix and other useful algorithms. It's your turn to solve the problem which Spotify, Amazon, and Yahoo Music faced every day.

*- Overview *

trainIterm2.txt - the training set

testIterm2.txt - the test set

sample_ submission.csv - a sample submission file in the correct format

trackData2.txt -- Track information formatted as:

<'TrackId'>|<'AlbumId'>|<'ArtistId'>|<'Optional GenreId_1'>|...|<'Optional GenreId_k'>

albumData2.txt -- Album information formatted as:

<'AlbumId'>|<'ArtistId'>|<'Optional GenreId_1'>|...|<'Optional GenreId_k'>

artistData2.txt -- Artist listing formatted as:

<'ArtistId'>

genreData2.txt -- Genre listing formatted as:

<'GenreId'>

In [47]:
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Set
from collections import OrderedDict
from scipy.sparse import csr_matrix

In [64]:
user_seperator = '|'

def load_training_dataset(file_path: str) -> csr_matrix:
    """
    returns a sparse matrix with columns of user ids and rows of track ids
    cell values are the rating for the given track
    """
    all_tracks: Set[int] = set()
    all_users: Set[int] = set()
    with open(file_path) as file_data:
        lines_remaining: int = 0
        current_user: Optional[int] = None
        for line in file_data:
            line = line.strip()
            if lines_remaining == 0:
                if user_seperator not in line:
                    raise ValueError('cannot find user seperator')
                current_user, lines_remaining = map(int, line.split(user_seperator))
                all_users.add(current_user)
            else:
                track_id, _score = map(int, line.split())
                all_tracks.add(track_id)
                lines_remaining -= 1

        all_track_indexes = {k: v for v, k in enumerate(sorted(list(all_tracks)))}

        output = csr_matrix((len(all_tracks), len(all_users)), dtype=np.uint8)

        lines_remaining = 0
        user_index: int = -1
        for line in file_data:
            line = line.strip()
            if lines_remaining == 0:
                if user_seperator not in line:
                    raise ValueError('cannot find user seperator')
                _current_user, lines_remaining = map(int, line.split(user_seperator))
                user_index += 1
            else:
                track_id, score = map(int, line.split())
                track_index = all_track_indexes[track_id]
                output[track_index][user_index] = score
                lines_remaining -= 1

        return output

training_data = load_training_dataset('data/trainItem2.txt')

In [73]:
TOL=1e-10
MAX_ITERS = 3

def converged(Z, d_norm):
    err = np.linalg.norm(Z, 'fro') / d_norm
    print('error: ', err)
    return err < TOL

def shrink(M, tau):
    S = np.abs(M) - tau
    return np.sign(M) * np.where(S>0, S, 0)

def _svd(M, rank): 
    return fbpca.pca(M, k=min(rank, np.min(M.shape)), raw=True)

def norm_op(M): 
    return _svd(M, 1)[1][0]

def svd_reconstruct(M, rank, min_sv):
    u, s, v = _svd(M, rank)
    s -= min_sv
    nnz = (s > 0).sum()
    return u[:,:nnz] @ np.diag(s[:nnz]) @ v[:nnz], nnz

def pcp(X, maxiter=10, k=10):
    m, n = X.shape
    trans = m < n

    if trans: 
        X = X.T; m, n = X.shape

    lamda = 1/np.sqrt(m)

    op_norm = norm_op(X)

    Y = np.copy(X) / max(op_norm, np.linalg.norm( X, np.inf) / lamda)

    mu = k*1.25/op_norm; mu_bar = mu * 1e7; rho = k * 1.5

    d_norm = np.linalg.norm(X, 'fro')
    L = np.zeros_like(X); sv = 1
    for i in range(maxiter):
        print("rank sv:", sv)
        X2 = X + Y/mu

        # update estimate of Sparse Matrix by "shrinking/truncating": original - low-rank
        S = shrink(X2 - L, lamda/mu)

        # update estimate of Low-rank Matrix by doing truncated SVD of rank sv & reconstructing.
        # count of singular values > 1/mu is returned as svp
        L, svp = svd_reconstruct(X2 - S, sv, 1/mu)

        # If svp < sv, you are already calculating enough singular values.
        # If not, add 20% (in this case 240) to sv
        sv = svp + (1 if svp < sv else round(0.05*n))

        # residual
        Z = X - L - S
        Y += mu * Z
        mu *= rho

        if m > mu_bar:
            m = mu_bar

        if converged(Z, d_norm): 
            break

    if trans:
        L=L.T; S=S.T

    return L, S

In [ ]:
# L, S = pcp(training_data)
import delayedsparse
pca=delayedsparse.PCA(n_components=5)
pca.fit(training_data)
Xmaped=pca.transform(training_data)
print(Xmaped.shape)

In [72]:
from sklearn.decomposition import SparsePCA
transformer = SparsePCA(n_components=5, random_state=0)
transformer.fit(training_data.toarray())
# rpca = R_pca(training_data)

MemoryError: Unable to allocate 108. GiB for an array with shape (295799, 49204) and data type float64

In [25]:
user_seperator = '|'

user_data: Dict[int, List[int]] = OrderedDict()
current_user: Optional[int] = None
lines_remaining = 0

with open('data/testItem2.txt') as test_data:
    for line in test_data:
        if lines_remaining == 0:
            if user_seperator not in line:
                raise ValueError('cannot find user seperator')
            current_user, lines_remaining = map(int, line.split(user_seperator))
        else:
            if current_user not in user_data:
                user_data[current_user] = np.array([], dtype=np.int)
            user_data[current_user] = np.append(user_data[current_user], int(line.strip()))
            lines_remaining -= 1

print(f'num users: {len(user_data)}')

num users: 20000


In [39]:
num_pick = 3

selections: Dict[int, List[int]] = OrderedDict()

for user_id, options in user_data.items():
    indexes = np.random.choice(options.shape[0], size=num_pick, replace=False)
    selections[user_id] = options[indexes]

print(f'num selections per user: {selections[user_id].shape}')

num selections per user: (3,)


In [45]:
user_track_ids: List[str] = []
predictors: List[int] = []

for user_id, tracks in user_data.items():
    for track_id in tracks:
        user_track_ids.append(f'{user_id}_{track_id}')
        predictors.append(1 if track_id in selections[user_id] else 0)

output_df = pd.DataFrame({'TrackID': user_track_ids, 'Predictor': predictors}).set_index('TrackID')
output_df.to_csv('data/output.csv')